<a href="https://colab.research.google.com/github/fertorresfs/Pipeline-Automatizado-de-Limpeza-de-Dados/blob/main/pipeline_limpeza_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import logging

# Configurar o logging
logging.basicConfig(filename='limpeza_dados.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
def carregar_dados(caminho_arquivo):
    """Carrega os dados de um arquivo CSV."""
    try:
        df = pd.read_csv(caminho_arquivo)
        logging.info(f"Dados carregados com sucesso de {caminho_arquivo}")
        return df
    except FileNotFoundError:
        logging.error(f"Arquivo não encontrado: {caminho_arquivo}")
        return None
    except Exception as e:
        logging.error(f"Erro ao carregar os dados: {e}")
        return None

In [ ]:
def tratar_valores_ausentes(df, estrategia='remover_linhas'):
    """Trata valores ausentes no DataFrame."""
    if estrategia == 'remover_linhas':
        df.dropna(inplace=True)
        logging.info("Valores ausentes removidos (linhas)")
    elif estrategia == 'preencher_media':
        for col in df.select_dtypes(include=np.number): #só numéricas
            df[col].fillna(df[col].mean(), inplace=True)
        logging.info("Valores ausentes preenchidos com a média")
    # Adicione mais estratégias aqui, como preencher com mediana, moda, etc.
    return df


In [ ]:
def formatar_dados(df, colunas_data=None, colunas_categoricas=None):
    """Formata colunas de data e categóricas."""
    if colunas_data:
        for col in colunas_data:
            try:
                df[col] = pd.to_datetime(df[col])
                logging.info(f"Coluna {col} formatada como data")
            except:
                logging.error(f"Erro ao formatar a coluna {col} como data")

    if colunas_categoricas:
        for col in colunas_categoricas:
            df[col] = df[col].astype('category')
            logging.info(f"Coluna {col} formatada como categórica")

    return df

In [ ]:
def detectar_outliers(df, colunas_numericas, metodo='zscore', limiar=3):
  """Detecta outliers em colunas numéricas usando Z-score ou IQR."""
  if metodo == 'zscore':
    for col in colunas_numericas:
        z = np.abs((df[col] - df[col].mean()) / df[col].std())
        df = df[(z < limiar)] #remove outliers
    logging.info(f"Outliers detectados e removidos (Z-score) nas colunas {colunas_numericas}")


  #implementação IQR omitida para simplificar, mas essencial num projeto real.
  return df

In [ ]:
# Exemplo de uso
caminho_arquivo = 'seu_arquivo.csv' # Substitua pelo caminho do seu arquivo
df = carregar_dados(caminho_arquivo)

In [ ]:
if df is not None:
    df = tratar_valores_ausentes(df) #ou df = tratar_valores_ausentes(df, estrategia='preencher_media')
    df = formatar_dados(df, colunas_data=['data_coluna'], colunas_categoricas=['categoria_coluna'])
    df = detectar_outliers(df, colunas_numericas=['coluna_numerica1', 'coluna_numerica2'])

    print(df.head()) #visualização
    df.to_csv('dados_limpos.csv', index=False)
    logging.info("Dados limpos salvos em dados_limpos.csv")